# MLM Training With Trainer

In [5]:
from transformers import BertTokenizer, BertForMaskedLM
import torch 

# load pre-train model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# load data
with open('../data/text/meditations/clean.txt', 'r') as f:
    text = f.read().split('\n')
text[:5]

['From my grandfather Verus I learned good morals and the government of my temper.',
 'From the reputation and remembrance of my father, modesty and a manly character.',
 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.',
 'From my great-grandfather, not to have frequented public schools, and to have had good teachers at home, and to know that on such things a man should spend liberally.',
 "From my governor, to be neither of the green nor of the blue party at the games in the Circus, nor a partizan either of the Parmularius or the Scutarius at the gladiators' fights; from him too I learned endurance of labour, and to want little, and to work with my own hands, and not to meddle with other people's affairs, and not to be ready to listen to slander."]

* Step
    * First, we'll tokenize our text.
    * create labels from input_ids


In [7]:
inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
inputs['labels'] = inputs.input_ids.detach().clone()

In [8]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [9]:
rand = torch.rand(inputs.input_ids.shape)
mask_arr = (rand<0.15) & (inputs.input_ids!=101) & \
    (inputs.input_ids!=102) & (inputs.input_ids!=0)

# get mask index form mask_arr

selections = []
for i in range(inputs.input_ids.shape[0]):
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    selections.append(selection)
    inputs.input_ids[i, selection] = 103 # mask input


The `Trainer` expects a `Dataset` object, so we need to initialize this.

In [11]:
class MLMDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return len(self.encodings.input_ids)
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}

dataset = MLMDataset(inputs)

# Training MLM Model by transformers's Trainer

In [12]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir='out',
    per_device_train_batch_size=16,
    num_train_epochs=2
)
trainer = Trainer(
    model = model,
    args=args,
    train_dataset=dataset
    )

In [ ]:
trainer.train()